In [2]:
import pandas as pd
import re

df = pd.read_csv("/content/complete_tree_species_data.csv")
def parse_range(value):
    if isinstance(value, str):
        value = value.replace("%", "").strip()
        match = re.match(r"(\d+\.?\d*)-(\d+\.?\d*)", value)
        if match:
            return float(match.group(1)), float(match.group(2))
    return None, None

df["pH_min"], df["pH_max"] = zip(*df["pH"].map(parse_range))
df["Humidity_min"], df["Humidity_max"] = zip(*df["Humidity"].map(parse_range))

df["pH_mid"] = (df["pH_min"] + df["pH_max"]) / 2
df["Humidity_mid"] = (df["Humidity_min"] + df["Humidity_max"]) / 2

def recommend_trees(ph, humidity, moisture, tree_type=None, min_lifespan=20, include_type=False):
    df_filtered = df[
        (df["Moisture"].str.lower() == moisture.lower()) &
        (df["Lifespan"] >= min_lifespan)
    ].copy()

    if tree_type and tree_type.strip():
        df_filtered = df_filtered[df_filtered["Type"].str.lower() == tree_type.lower()]

    df_filtered["score"] = ((df_filtered["pH_mid"] - ph)**2 +
                            (df_filtered["Humidity_mid"] - humidity)**2) ** 0.5

    df_filtered = df_filtered.sort_values("score")

    if include_type:
        result = df_filtered[["Name", "Lifespan", "Type"]]
    else:
        result = df_filtered[["Name", "Lifespan"]]

    return result